In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Heart_rate/GSE18583'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Baseline skeletal muscle gene expression"
!Series_summary	"Muscle biopsy samples were obtained from two groups of male subjects prior to endurance training. The samples were used to predict training responses."
!Series_summary	"Baseline gene expression involving 30 probe sets was able to classify subjects into high and low responders."
!Series_overall_design	"Resting skeletal muscle sample after an overnight fast."
Sample Characteristics Dictionary:
{0: ['gender: male'], 1: ['protocol: Resting skeletal muscle sample prior to endurance training'], 2: ['heart rate (bpm): 173', 'heart rate (bpm): 155', 'heart rate (bpm): 183', 'heart rate (bpm): 149', 'heart rate (bpm): 146', 'heart rate (bpm): 157', 'heart rate (bpm): 162', 'heart rate (bpm): 170', 'heart rate (bpm): 165', 'heart rate (bpm): 144', 'heart rate (bpm): 167', 'heart rate (bpm): 191', 'heart rate (bpm): 160', 'heart rate (bpm): 177', 'heart rate (bpm): 174', 'heart rate (bpm): 190', 'hea

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True  # It appears to contain gene expression data

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
trait_row = 2  # heart rate data is available in row 2
age_row = None  # age data is not available
gender_row = 0  # gender data is available in row 0

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        # Extract value after colon and convert to float
        return float(value.split(': ')[1])
    except:
        return None

def convert_age(value):
    try:
        # Extract value after colon and convert to float (if age data were available)
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    try:
        # Convert male to 1 and female to 0
        gender_str = value.split(': ')[1].strip().lower()
        if gender_str == 'male':
            return 1
        elif gender_str == 'female':
            return 0
        else:
            return None
    except:
        return None

# 3. Save Metadata
save_cohort_info('GSE18583', './preprocessed/Heart_rate/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Heart_rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Heart_rate/trait_data/GSE18583.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM462215': [173.0, 1.0], 'GSM462216': [155.0, 1.0], 'GSM462217': [183.0, 1.0], 'GSM462218': [149.0, 1.0], 'GSM462219': [146.0, 1.0], 'GSM462220': [157.0, 1.0], 'GSM462221': [162.0, 1.0], 'GSM462222': [170.0, 1.0], 'GSM462223': [165.0, 1.0], 'GSM462224': [144.0, 1.0], 'GSM462225': [167.0, 1.0], 'GSM462226': [165.0, 1.0], 'GSM462227': [191.0, 1.0], 'GSM462228': [149.0, 1.0], 'GSM462229': [160.0, 1.0], 'GSM462230': [177.0, 1.0], 'GSM462231': [174.0, 1.0], 'GSM462232': [165.0, 1.0], 'GSM462233': [173.0, 1.0], 'GSM462234': [190.0, 1.0], 'GSM462235': [190.0, 1.0], 'GSM462236': [190.0, 1.0], 'GSM462237': [169.0, 1.0], 'GSM462238': [160.0, 1.0], 'GSM462239': [None, 1], 'GSM462240': [None, 1], 'GSM462241': [None, 1], 'GSM462242': [None, 1], 'GSM462243': [None, 1], 'GSM462244': [None, 1], 'GSM462245': [None, 1], 'GSM462246': [None, 1], 'GSM462247': [None, 1], 'GSM462248': [None, 1], 'GSM462249': [None, 1], 'GSM462250': [None, 1], 'GSM462251': [None, 1], 'GSM462252': [None, 1], 'GSM462253': [N

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ENST00000000233_at', 'ENST00000000412_at', 'ENST00000000442_at',
       'ENST00000001008_at', 'ENST00000002125_at', 'ENST00000002165_at',
       'ENST00000002501_at', 'ENST00000002829_at', 'ENST00000003100_at',
       'ENST00000003302_at', 'ENST00000003583_at', 'ENST00000003607_at',
       'ENST00000003912_at', 'ENST00000004531_at', 'ENST00000004921_at',
       'ENST00000004980_at', 'ENST00000004982_at', 'ENST00000005082_at',
       'ENST00000005178_at', 'ENST00000005198_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ENST00000000233_at', 'ENST00000000412_at', 'ENST00000000442_at', 'ENST00000001008_at', 'ENST00000002125_at'], 'SPOT_ID': ['ENST00000000233', 'ENST00000000412', 'ENST00000000442', 'ENST00000001008', 'ENST00000002125']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying respective keys
identifier_key = 'ID'
gene_symbol_key = 'SPOT_ID'

# 2. Getting the DataFrame storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Applying the mapping to get the gene expression dataframe named "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)
